# Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '../')
from plot_graphs import plot_loss, plot_acc

# Setting Image and Batch size

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

# Importing the train, test, and val data

In [3]:
train = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)


Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-04 06:01:04.822877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-04 06:01:04.823008: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Preprocessing the data for VGG16

In [4]:
train = train.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
val = val.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
test = test.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))

# Defining the model

In [5]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling='avg',
    classes=6,
    classifier_activation="softmax"
)

model.compile(optimizer=keras.optimizers.RMSprop(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

# Running the Model

In [6]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../models/sota/sota_vgg16.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=20,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/20


2022-08-04 06:01:05.211290: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-04 06:01:05.829007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.4288 - accuracy: 0.4869

2022-08-04 06:01:59.519935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 57s 538ms/step - loss: 1.4288 - accuracy: 0.4869 - val_loss: 0.6312 - val_accuracy: 0.7653
Epoch 2/20
105/105 [==============================] - 55s 527ms/step - loss: 0.5857 - accuracy: 0.7798 - val_loss: 0.4381 - val_accuracy: 0.8347
Epoch 3/20
105/105 [==============================] - 55s 520ms/step - loss: 0.3492 - accuracy: 0.8777 - val_loss: 0.4972 - val_accuracy: 0.8500
Epoch 4/20
105/105 [==============================] - 55s 522ms/step - loss: 0.2521 - accuracy: 0.9155 - val_loss: 0.3748 - val_accuracy: 0.8972
Epoch 5/20
105/105 [==============================] - 55s 523ms/step - loss: 0.1752 - accuracy: 0.9405 - val_loss: 0.3623 - val_accuracy: 0.9111
Epoch 6/20
105/105 [==============================] - 56s 532ms/step - loss: 0.1344 - accuracy: 0.9497 - val_loss: 0.2997 - val_accuracy: 0.9250
Epoch 7/20
105/105 [==============================] - 57s 541ms/step - loss: 0.1158 - accuracy: 0.9649 - val_loss: 0.2874 - val_accuracy: 0.9

# Evaluating the model

In [7]:
model.evaluate(test)

load_model = keras.models.load_model("../models/sota/sota_vgg16.h5")
load_model.evaluate(test)

23/23 [==============================] - 3s 130ms/step - loss: 0.3191 - accuracy: 0.9375


2022-08-04 06:20:35.656830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 3s 129ms/step - loss: 0.2512 - accuracy: 0.9583


[0.25120633840560913, 0.9583333730697632]

In [ ]:
# Plotting loss and accuracy graphs
## Plotting loss
plot_loss(history)
## Plotting accuracy
plot_acc(history)